# Information retrieval

## Home Work 6

## Evaluating IR Model
### Group 1
#### Ali Mojahed - 9812762554
#### Mehrnoosh Navidimehr - 9822762119
#### Minoo Mohaghegh - 9812762270
#### Helia Ghahraman - 9822762437



# Download Datasets

In [1]:
!wget http://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz

--2023-06-09 18:42:28--  http://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz
Resolving ir.dcs.gla.ac.uk (ir.dcs.gla.ac.uk)... 130.209.240.253
Connecting to ir.dcs.gla.ac.uk (ir.dcs.gla.ac.uk)|130.209.240.253|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 775144 (757K) [application/gzip]
Saving to: ‘cisi.tar.gz’

cisi.tar.gz         100%[===================>] 756.98K  1.29MB/s    in 0.6s    

2023-06-09 18:42:29 (1.29 MB/s) - ‘cisi.tar.gz’ saved [775144/775144]



In [2]:
!tar xvzf cisi.tar.gz

CISI.ALL
CISI.BLN
CISI.QRY
CISI.REL


# Import Required Libraries

In [3]:
import nltk
import numpy as np
import re
from nltk.stem import PorterStemmer


In [4]:
my_stopwords = ['ourselves','hers','between','yourself','but','again','there','about','once','during','out','very','having','with',
'they','own','an','be','some','for','do','its','yours','such','into','of','most','itself','other','off','is','s',
'am','or','who','as','from','him','each','the','themselves','until','below','are','we','these','your','his','through',
'don','nor','me','were','her','more','himself','this','down','should','our','their','while','above','both','up','to',
'ours','had','she','all','no','when','at','any','before','them','same','and','been','have','in','will','on','does',
'yourselves','then','that','because','what','over','why','so','can','did','not','now','under','he','you','herself','has',
'just','where','too','only','myself','which','those','i','after','few','whom','t','being','if','theirs','my','against','a',
'by','doing','it','how','further','was','here','than']

# Parse Dataset

In [5]:
def stemming(word):
    stemmer = PorterStemmer()
    stemmed_word = stemmer.stem(word)
    return stemmed_word

def clean_doc(liste):
    res = []
    for word in liste :
        new_word = word.lower()
        new_word = re.sub(r'[^\w\s]', '', new_word)
        if new_word != "":
            res.append(new_word)
    return res

def clean_doc_stem(liste):
    res = []
    for word in liste :
        new_word = word.lower()
        new_word = re.sub(r'[^\w\s]', '', new_word)
        if new_word != "":
            new_word_stem = stemming(new_word)
            res.append(new_word_stem)
    return res

In [6]:
def indexation(document):
    words = document.split(' ')
    words = clean_doc(words)
    doc_len = len(words)
    word_count = {}
    for word in words :
        word_count[word] = 0
    for word in words :
        word_count[word] += 1
    
    return word_count, doc_len

In [7]:
def indexation_stem(document):
    words = document.split(' ')
    words = clean_doc_stem(words)
    doc_len = len(words)
    word_count = {}
    for word in words :
        word_count[word] = 0
    for word in words :
        word_count[word] += 1
    
    return word_count, doc_len

In [8]:
def loadAll(filename, stem) :
    word_set = []
    cisi_all = {}
    id = 0
    lines = open(filename).read()
    for document in lines.split(".I ") :
        token = ""
        buffer = ""
        cross_reference = []
        cisi_all[id] = {}
        cisi_all[id]['ID'] = id
        cisi_all[id]['title'] = ""
        cisi_all[id]["text"] = ""
        document += "EndOfDocument"
        for line in document.split("\n") :
            if token == ".T" and line != ".A":
                cisi_all[id]['title'] = line.replace("\n", " ")
            elif token == ".W" and line != ".X" :
                buffer += " " + line.replace("\n", " ")
                continue
            elif line == ".X":
                cisi_all[id]["text"] = buffer
                buffer = ""
            elif token == ".X" and line != "EndOfDocument":
                cross_reference.append(line.split("\t"))
                continue
            elif line == "EndOfDocument":
                cisi_all[id]["cross_ref"] = cross_reference
                cross_reference = []
            token = line
        if(stem == True):
            cisi_all[id]['word_count'], cisi_all[id]['doc_len'] = indexation_stem(cisi_all[id]["text"] + " " + cisi_all[id]['title'])
        else :
            cisi_all[id]['word_count'], cisi_all[id]['doc_len'] = indexation(cisi_all[id]["text"] + " " + cisi_all[id]['title'])
        for word in cisi_all[id]['word_count'].keys():
            if word not in word_set :
                word_set.append(word)
        id += 1
    del cisi_all[0]
    return cisi_all

In [33]:
documents = loadAll("CISI.ALL", False)

In [ ]:
documents

In [12]:
def loadQuery(filename, stem) :
    cisi_qry = {}
    id = 0
    lines = open(filename).read()
    for document in lines.split(".I ") :
        token = ""
        buffer = ""
        cisi_qry[id] = {}
        cisi_qry[id]['ID'] = id
        cisi_qry[id]["query"] = ""
        cisi_qry[id]["text"] = ""
        document += "EndOfDocument"
        for line in document.split("\n") :
            if token == ".W" and line != "EndOfDocument" :
                buffer += " " + line.replace("\n", " ")
                continue
            elif line == "EndOfDocument":
                cisi_qry[id]["query"] = buffer
                cisi_qry[id]["text"] = buffer
                buffer = ""
                
                continue
            token = line
        if(stem == True):
            cisi_qry[id]['query'], cisi_qry[id]['doc_len'] = indexation_stem(cisi_qry[id]["query"])
        else :
            cisi_qry[id]['query'], cisi_qry[id]['doc_len'] = indexation(cisi_qry[id]["query"])
        id += 1
    del cisi_qry[0]
    return cisi_qry

In [13]:
def loadRel(filename) : 
    cisi_rel = {}
    lines = open(filename).read()
    lines += "EndOfDocument"
    buffer = 1
    list_buffer = []
    for line in lines.split("\n") :
        if(line == "EndOfDocument"):
            cisi_rel[token] = list_buffer
            return cisi_rel
        else :
            token = int((line[0:6]).replace(" ",""))
            if(token == buffer):
                list_buffer.append(int((line[7:13]).replace(" ","")))
                buffer = token
                continue
            else :
                cisi_rel[buffer] = list_buffer
                list_buffer = []
                list_buffer.append(int((line[7:13]).replace(" ","")))
                buffer = token
                continue

In [14]:
queries = loadQuery("CISI.QRY", False)

In [ ]:
queries

In [16]:
relations = loadRel("CISI.REL")

# Preprocess

In [17]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [44]:
def preprocess_text(text, stem=False, remove_stopwords=False):
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word.lower() for word in tokens]
    
    if remove_stopwords:
      filtered_tokens = [word for word in filtered_tokens if word.lower() not in stop_words]

    # Perform stemming if stem=True
    if stem:
        stemmer = PorterStemmer()
        stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
        return stemmed_tokens
    
    return filtered_tokens

# Posting-List

In [20]:
def create_posting_list(documents, stem=False, remove_stopwords=True):
    posting_list = {}
    
    for doc_id, document in documents.items():
        text = document['text']
        word_count = document['word_count']
        
        # Preprocess text
        tokens = preprocess_text(text, stem, remove_stopwords)
        
        # Update posting list
        for token in tokens:
            if token in posting_list:
                posting_list[token].append((doc_id, word_count.get(token, 0)))
            else:
                posting_list[token] = [(doc_id, word_count.get(token, 0))]
    
    return posting_list

# Information Retreival

In [82]:
import math
import numpy as np

def retrieve_documents(method, query, documents, stem=False, remove_stopwords=True):
    """
    Retrieve and rank documents based on the specified retrieval method.
    
    Args:
        method (str): Retrieval method string ('lnc.ltc', 'ltc.ltc', 'lnn.lnn', 'lnc.lnc', 'ltn.ltn', 'bnn.bnn').
        query (str): Query string.
        documents (dict): Dictionary of document IDs and corresponding objects.
    
    Returns:
        list: List of top-ranked document IDs.
    """
    # Tokenize query
    query_tokens = preprocess_text(query['text'], stem, remove_stopwords)
    # print(query_tokens)
    # Create a dictionary to store the document scores
    # Create a dictionary to store the document scores
    document_scores = {}
    
    # Create arrays to store the IDF scores and document frequencies
    query_idf_scores = np.array([np.log(len(documents) / (sum(1 for doc in documents.values() if term in doc['token_count']) + 1)) for term in query_tokens])
    doc_frequencies = np.array([sum(1 for doc in documents.values() if term in doc['token_count']) for term in query_tokens])
    
    # Calculate document scores based on the retrieval method
    for doc_id, doc in documents.items():
        term_frequencies = np.array([doc['token_count'].get(term, 0) for term in query_tokens])
        if term_frequencies == 0:
          term_frequencies = np.ones_like(term_frequencies)

        if method == 'lnc.ltc':
            max_term_frequencies = np.array([max(doc['token_count'].values(), default=1) for _ in query_tokens])

            if max_term_frequencies == 0:
              max_term_frequencies = 1

            document_weights = (1 + np.log(term_frequencies)) / (1 + np.log(max_term_frequencies))
            
        elif method == 'ltc.ltc':
            document_weights = term_frequencies * query_idf_scores
            
        elif method == 'lnn.lnn':
            document_weights = (1 + np.log(term_frequencies)) * query_idf_scores
            
        elif method == 'lnc.lnc':
            max_term_frequencies = np.array([max(doc['token_count'].values(), default=1) for _ in query_tokens])
            if max_term_frequencies == 0:
              max_term_frequencies = 1

            document_weights = ((1 + np.log(term_frequencies)) / (1 + np.log(max_term_frequencies))) * query_idf_scores
            
        elif method == 'ltn.ltn':
            max_term_frequencies = np.array([max(doc['token_count'].values(), default=1) for _ in query_tokens])
            if max_term_frequencies == 0:
              max_term_frequencies = 1

            document_weights = ((1 + np.log(term_frequencies)) * query_idf_scores) / (1 + np.log(max_term_frequencies))
            
        elif method == 'bnn.bnn':
            document_weights = np.array([1 if term in doc['token_count'] else 0 for term in query_tokens])
        
        # Calculate the document score based on the weights
        document_score = np.sum(document_weights)
        document_scores[doc_id] = document_score
    
    # Rank the documents based on their scores
    ranked_documents = sorted(document_scores, key=document_scores.get, reverse=True)
    
    return ranked_documents


# Evaluations

In [23]:
def precision_at_k(ground_truth, predicted, k):
    """
    Compute precision@k.
    
    Args:
        ground_truth (list): List of ground truth document IDs.
        predicted (list): List of predicted document IDs.
        k (int): Value of k for precision@k.
    
    Returns:
        float: Precision@k score.
    """
    # print(ground_truth, predicted)
    if len(predicted) < k:
        k = len(predicted)
    
    if k == 0:
        return 0.0
    
    num_correct = len(set(predicted[:k]).intersection(set(ground_truth)))
    precision = num_correct / k
    
    return precision

def recall_at_k(ground_truth, predicted, k):
    """
    Compute recall@k.
    
    Args:
        ground_truth (list): List of ground truth document IDs.
        predicted (list): List of predicted document IDs.
        k (int): Value of k for recall@k.
    
    Returns:
        float: Recall@k score.
    """
    if len(predicted) < k:
        k = len(predicted)
    
    if len(ground_truth) == 0:
        return 0.0
    
    num_correct = len(set(predicted[:k]).intersection(set(ground_truth)))
    recall = num_correct / len(ground_truth)
    
    return recall

def f1_score(precision, recall):
    """
    Compute F1-score.
    
    Args:
        precision (float): Precision value.
        recall (float): Recall value.
    
    Returns:
        float: F1-score.
    """
    if precision == 0 or recall == 0:
        return 0.0
    
    f1 = (2 * precision * recall) / (precision + recall)
    
    return f1

def average_precision(ground_truth, predicted):
    """
    Compute Average Precision (AP).
    
    Args:
        ground_truth (list): List of ground truth document IDs.
        predicted (list): List of predicted document IDs.
    
    Returns:
        float: Average Precision (AP) score.
    """
    num_correct = 0
    precision_sum = 0.0
    
    for i, doc_id in enumerate(predicted):
        if doc_id in ground_truth:
            num_correct += 1
            precision = num_correct / (i + 1)
            precision_sum += precision
    
    if num_correct == 0:
        return 0.0
    
    ap = precision_sum / num_correct
    
    return ap

def mean_average_precision(query_ground_truth, query_predictions):
    """
    Compute Mean Average Precision (MAP).
    
    Args:
        query_ground_truth (dict): Dictionary of query IDs and corresponding ground truth document IDs.
        query_predictions (dict): Dictionary of query IDs and corresponding predicted document IDs.
    
    Returns:
        float: Mean Average Precision (MAP) score.
    """
    num_queries = len(query_ground_truth)
    map_sum = 0.0
    
    for query_id, ground_truth in query_ground_truth.items():
        predicted = query_predictions.get(query_id, [])
        ap = average_precision(ground_truth, predicted)
        map_sum += ap
    
    map_score = map_sum / num_queries
    
    return map_score

def discounted_cumulative_gain(ground_truth, predicted, k):
    """
    Compute Discounted Cumulative Gain (DCG).
    
    Args:
        ground_truth (list): List of ground truth document IDs.
        predicted (list): List of predicted document IDs.
        k (int): Value of k for DCG.
    
    Returns:
        float: Discounted Cumulative Gain (DCG) score.
    """
    if len(predicted) < k:
        k = len(predicted)
    
    dcg = 0.0
    
    for i in range(k):
        doc_id = predicted[i]
        if doc_id in ground_truth:
            relevance = 1
        else:
            relevance = 0
        
        gain = 2 ** relevance - 1
        discount = 1 / (i + 1)
        dcg += gain * discount
    
    return dcg




In [24]:
posting_list = create_posting_list(documents, True)

# Evaluation

In [88]:
def compute_for_each_method(method, stem, allow_stop_words):
  query_prediction = {}
  query_precisions = []
  query_recalls = []
  query_f1_scores = []
  query_aps = []
  query_dcgs = []
  # posting_list = create_posting_list(documents, stem, allow_stop_words)
  for doc_id, doc in documents.items():
    documents[doc_id]['tokens'] = preprocess_text(doc["text"], stem, allow_stop_words)
    documents[doc_id]['token_count'] = {key: value for key, value in doc['word_count'].items() if key in documents[doc_id]['tokens']}
    # print(documents[doc_id]['token_count'])
  
  k = 20
  for query_id, query in queries.items():
    if query_id not in relations.keys():
      continue
    query_prediction[query_id] = retrieve_documents(method, query, documents, stem, allow_stop_words)
    precision = precision_at_k(relations[query_id], query_prediction[query_id], k)
    recall = recall_at_k(relations[query_id], query_prediction[query_id], k)
    f1 = f1_score(precision, recall)
    map_score = mean_average_precision(relations, query_prediction)
    dcg = discounted_cumulative_gain(relations[query_id], query_prediction[query_id], k)
    
    query_precisions.append(precision)
    query_recalls.append(recall)
    query_f1_scores.append(f1)
    query_aps.append(map_score)
    query_dcgs.append(dcg)
    # print(query_id)



  overall_precision = sum(query_precisions) / len(query_precisions)
  overall_recall = sum(query_recalls) / len(query_recalls)
  overall_f1_score = sum(query_f1_scores) / len(query_f1_scores)
  overall_map = sum(query_aps) / len(query_aps)
  overall_dcg = sum(query_dcgs) / len(query_dcgs)

  print(f"Method: {method}, stem {stem}, stopwords {allow_stop_words}")
  print(f'Overall Precision: {overall_precision}')
  print(f'Overall Recall: {overall_recall}')
  print(f'Overall F1-score: {overall_f1_score}')
  print(f'Overall MAP: {overall_map}')
  print(f'Overall DCG: {overall_dcg}')

  return overall_precision, overall_recall, overall_f1_score, overall_map, overall_dcg


# Report

In [89]:
from tabulate import tabulate
import itertools
# Define the list of methods, stopwords, and stemming options
methods = [ 'lnc.ltc', 'ltc.ltc', 'lnn.lnn', 'lnc.lnc', 'ltn.ltn', 'bnn.bnn']
bool_comb = [(True, False), (False, True), (False, False), (True, True)]
stemming_options = [True, False]

# Define the list of evaluation metrics
metrics = ['Precision', 'Recall', 'F1-score', 'MAP', 'DCG']

# Create an empty list to store the table data
table_data = []

# Perform the experiment for each method, stopwords option, and stemming option
# for method, allow_stopwords, stemming in zip(methods, stopwords_options, stemming_options):
for method, bools in itertools.product(methods, bool_comb):
    allow_stopwords, stemming = bools
    # # Perform the retrieval and evaluation
    # top_documents = retrieve_documents(method, query, documents, stemming, stopwords)
    # precision = precision_at_k(ground_truth, top_documents, k)
    # recall = recall_at_k(ground_truth, top_documents, k)
    # f1 = f1_score(precision, recall)
    # map_score = mean_average_precision(query_ground_truth, query_predictions)
    # dcg = discounted_cumulative_gain(ground_truth, top_documents, k)
    print(method, allow_stopwords, stemming)
    precision, recall, f1, map_score, dcg = compute_for_each_method(method, stemming, allow_stopwords)
    
    # Append the results to the table data
    table_data.append([method, allow_stopwords, stemming, precision, recall, f1, map_score, dcg])

# Define the headers for the table
headers = ['Method', 'Stopwords', 'Stemming', 'Precision', 'Recall', 'F1-score', 'MAP', 'DCG']

# Print the table
print(tabulate(table_data, headers=headers, tablefmt='pipe'))

lnc.ltc True False


<ipython-input-82-ffde8b210369>:33: RuntimeWarning: divide by zero encountered in log
  document_weights = (1 + np.log(term_frequencies)) / (1 + np.log(max_term_frequencies))


Method: lnc.ltc, stem False, stopwords True
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
lnc.ltc False True
Method: lnc.ltc, stem True, stopwords False
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
lnc.ltc False False
Method: lnc.ltc, stem False, stopwords False
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
lnc.ltc True True
Method: lnc.ltc, stem True, stopwords True
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
ltc.ltc True False
Method: ltc.ltc, stem False,

<ipython-input-82-ffde8b210369>:39: RuntimeWarning: divide by zero encountered in log
  document_weights = (1 + np.log(term_frequencies)) * query_idf_scores


Method: lnn.lnn, stem False, stopwords True
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
lnn.lnn False True
Method: lnn.lnn, stem True, stopwords False
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
lnn.lnn False False
Method: lnn.lnn, stem False, stopwords False
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
lnn.lnn True True
Method: lnn.lnn, stem True, stopwords True
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
lnc.lnc True False


<ipython-input-82-ffde8b210369>:43: RuntimeWarning: divide by zero encountered in log
  document_weights = ((1 + np.log(term_frequencies)) / (1 + np.log(max_term_frequencies))) * query_idf_scores


Method: lnc.lnc, stem False, stopwords True
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
lnc.lnc False True
Method: lnc.lnc, stem True, stopwords False
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
lnc.lnc False False
Method: lnc.lnc, stem False, stopwords False
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
lnc.lnc True True
Method: lnc.lnc, stem True, stopwords True
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
ltn.ltn True False


<ipython-input-82-ffde8b210369>:47: RuntimeWarning: divide by zero encountered in log
  document_weights = ((1 + np.log(term_frequencies)) * query_idf_scores) / (1 + np.log(max_term_frequencies))


Method: ltn.ltn, stem False, stopwords True
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
ltn.ltn False True
Method: ltn.ltn, stem True, stopwords False
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
ltn.ltn False False
Method: ltn.ltn, stem False, stopwords False
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
ltn.ltn True True
Method: ltn.ltn, stem True, stopwords True
Overall Precision: 0.0388157894736842
Overall Recall: 0.01723530105486803
Overall F1-score: 0.021951178388988572
Overall MAP: 0.022012196702679666
Overall DCG: 0.13538945811941572
bnn.bnn True False
Method: bnn.bnn, stem False,

In [90]:
print(tabulate(table_data, headers=headers, tablefmt='pipe'))

| Method   | Stopwords   | Stemming   |   Precision |    Recall |   F1-score |       MAP |      DCG |
|:---------|:------------|:-----------|------------:|----------:|-----------:|----------:|---------:|
| lnc.ltc  | True        | False      |   0.0388158 | 0.0172353 |  0.0219512 | 0.0220122 | 0.135389 |
| lnc.ltc  | False       | True       |   0.0388158 | 0.0172353 |  0.0219512 | 0.0220122 | 0.135389 |
| lnc.ltc  | False       | False      |   0.0388158 | 0.0172353 |  0.0219512 | 0.0220122 | 0.135389 |
| lnc.ltc  | True        | True       |   0.0388158 | 0.0172353 |  0.0219512 | 0.0220122 | 0.135389 |
| ltc.ltc  | True        | False      |   0.207895  | 0.143544  |  0.139546  | 0.07872   | 0.991267 |
| ltc.ltc  | False       | True       |   0.103289  | 0.0548318 |  0.0617196 | 0.0361119 | 0.429239 |
| ltc.ltc  | False       | False      |   0.175     | 0.116672  |  0.114239  | 0.0676422 | 0.789058 |
| ltc.ltc  | True        | True       |   0.126316  | 0.0654487 |  0.0753808 | 0.0